In [25]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from pprint import pprint

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\espoe\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


### Scrape the news site

In [3]:
url = 'http://redplanetscience.com'
browser.visit(url)
#optional delay for loading the page.  looks for tag/attrib combo and waits before query to ensure full page loads first
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [9]:
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"The Extraordinary Sample-Gathering System of NASA's Perseverance Mars Rover"

In [11]:
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.'

### Scrape the space image site

In [12]:
url = 'http://spaceimages-mars.com'
browser.visit(url)

In [13]:
# Find by tag is a different way to dig up tags!  neat.
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [14]:
# parse this page
html = browser.html
img_soup = soup(html, 'html.parser')

In [15]:
# grab relative url of first image
img_url_rel = img_soup.find('img', class_="fancybox-image").get("src")
img_url_rel

'image/featured/mars2.jpg'

In [17]:
img_url = f'{url}/{img_url_rel}'
img_url

'http://spaceimages-mars.com/image/featured/mars2.jpg'

In [23]:
# Read fact table using pandas read_html()
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns = ['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [26]:
# convert table back to html
pprint(df.to_html())

('<table border="1" class="dataframe">\n'
 '  <thead>\n'
 '    <tr style="text-align: right;">\n'
 '      <th></th>\n'
 '      <th>Mars</th>\n'
 '      <th>Earth</th>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>description</th>\n'
 '      <th></th>\n'
 '      <th></th>\n'
 '    </tr>\n'
 '  </thead>\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <th>Mars - Earth Comparison</th>\n'
 '      <td>Mars</td>\n'
 '      <td>Earth</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>Diameter:</th>\n'
 '      <td>6,779 km</td>\n'
 '      <td>12,742 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>Mass:</th>\n'
 '      <td>6.39 × 10^23 kg</td>\n'
 '      <td>5.97 × 10^24 kg</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>Moons:</th>\n'
 '      <td>2</td>\n'
 '      <td>1</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>Distance from Sun:</th>\n'
 '      <td>227,943,824 km</td>\n'
 '      <td>149,598,262 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>Length of Year:</th>\n'
 '      <td>687 Earth da

In [27]:
# close the durn session!
browser.quit()